In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import numpy as np
import os 
import glob
import cv2
import matplotlib.pyplot as plt
import datetime


#For Nueral Network
import tensorflow as tf
from tensorflow.keras.models import Sequential , load_model
from tensorflow.keras.layers import Conv2D , MaxPooling2D, AveragePooling2D , Dense , Flatten 

#For Image preprocessing
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.model_selection import train_test_split

import warnings as wr
wr.filterwarnings("ignore")

%matplotlib inline

In [ ]:
trainDataPath = "/content/train/"


In [ ]:
len(os.listdir(trainDataPath))

7421

In [ ]:
trainImagePath = os.path.join(trainDataPath, "*g")

In [ ]:
imageFile = glob.glob(trainImagePath)

In [ ]:
trainImagePath

'/content/train/*g'

In [ ]:
#imageFile

In [ ]:
def imageFeatureExtrac(image , size = (224,224)):
    
    img = cv2.resize(image , size)
   
    
    return img
    

In [ ]:
data = []

classes = []

for i , file in enumerate(imageFile):
    image = cv2.imread(file)
    feature = cv2.resize(image , (224,224))
    data.append(feature)
    classesNames = file.split(os.path.sep)[-1].split(".")[0]
    classes.append(classesNames)


In [ ]:
np.unique(classes)

array(['cat', 'dog'], dtype='<U3')

In [ ]:
imageData = np.array(data,dtype="uint8")

In [ ]:
labels = np.array(classes)

In [ ]:
imageData[0]


array([[[178, 199, 200],
        [170, 191, 192],
        [168, 189, 191],
        ...,
        [ 20,   7,  15],
        [ 26,  10,  10],
        [ 29,  13,  10]],

       [[182, 203, 204],
        [181, 202, 203],
        [175, 196, 198],
        ...,
        [ 18,   5,  13],
        [ 19,   5,   6],
        [ 23,   8,   6]],

       [[179, 200, 201],
        [176, 197, 198],
        [174, 196, 197],
        ...,
        [ 13,   3,  10],
        [ 12,   1,   2],
        [ 16,   3,   4]],

       ...,

       [[118, 122, 163],
        [125, 126, 170],
        [122, 118, 165],
        ...,
        [  7,   1,   2],
        [  6,   1,   2],
        [  4,   2,   2]],

       [[122, 126, 167],
        [128, 127, 171],
        [130, 124, 171],
        ...,
        [  7,   1,   2],
        [  6,   1,   2],
        [  4,   2,   2]],

       [[129, 131, 172],
        [130, 129, 173],
        [136, 130, 177],
        ...,
        [  7,   1,   2],
        [  6,   1,   2],
        [  4,   2,   2]]

In [ ]:
inputShape = imageData[0].shape
inputShape

(224, 224, 3)

In [ ]:
labelEncoderValues = LabelEncoder().fit_transform(labels)
np.unique(labelEncoderValues)

array([0, 1])

In [ ]:
labelEncoderValues = labelEncoderValues.reshape(len(labelEncoderValues),1)
classes = OneHotEncoder(sparse=False).fit_transform(labelEncoderValues)

In [ ]:
np.unique(classes)

array([0., 1.])

In [ ]:
print(f"Length of ImageData = {len(imageData)}\nLength of Classes = {len(classes)}")


Length of ImageData = 7420
Length of Classes = 7420


In [ ]:
#Separate Dataset

xtrain,xtest,ytrain,ytest = train_test_split(imageData , classes , test_size = 0.20 , random_state = 1)

In [ ]:
print(f"Length of Xtrain = {len(xtrain)}\nLength of Ytrain = {len(ytrain)}\nLength of Xtest = {len(xtest)}\nLength of ytest = {len(xtest)}")

Length of Xtrain = 5936
Length of Ytrain = 5936
Length of Xtest = 1484
Length of ytest = 1484


In [ ]:
#Model Architecture

class AlexNet(Sequential):

  def __init__(self , InputShape , NumberOfClass):
      super().__init__()

      #1st Layer
      self.add(Conv2D(filters = 96 , kernel_size = (11,11) , strides = (4,4) , padding = "valid" , activation ="relu" , input_shape = InputShape)),
      self.add(MaxPooling2D(pool_size=(3,3) , strides=(2,2) , padding = "valid" , data_format=None)),

      #2nd Layer
      self.add(Conv2D(filters = 256 , kernel_size = (5,5) , strides = 1 , padding = "same" , activation ="relu")),
      self.add(MaxPooling2D(pool_size=(3,3) , strides=(2,2) , padding = "valid" , data_format=None)),

      #3rd Layer
      self.add(Conv2D(filters = 384 , kernel_size = (3,3) , strides = 1 , padding = "same" , activation ="relu")),

      #4th Layer
      self.add(Conv2D(filters = 384 , kernel_size = (3,3) , strides = 1 , padding = "same" , activation ="relu")),

      #5th Layer
      self.add(Conv2D(filters = 384 , kernel_size = (3,3) , strides = 1 , padding = "same" , activation ="relu")),
      self.add(MaxPooling2D(pool_size=(3,3) , strides=(2,2) , padding = "valid" , data_format=None)),

      #Flatten Layer
      self.add(Flatten()),


      #Dense Layers
      self.add(Dense(4096,activation = "relu")),
      self.add(Dense(4096 , activation = "relu")),
      self.add(Dense(NumberOfClass , activation = "softmax")),

      #Model Compile
      self.compile(
        optimizer = "adam",
        loss = "categorical_crossentropy",
        metrics = ["accuracy"]
      )





In [ ]:
numOfClass = len(np.unique(classes))

In [ ]:
model = AlexNet(inputShape , numOfClass)

In [ ]:
model.summary()

Model: "alex_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 384)       885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 384)       132

In [ ]:
logsData = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
callbacks = tf.keras.callbacks.TensorBoard(log_dir = logsData , histogram_freq = 1)

In [ ]:

history = model.fit(
    
    xtrain , ytrain,
    validation_data = (xtest , ytest) , 
    callbacks = [callbacks],
    epochs = 1,
    verbose = 1)

186/186 [==============================] - ETA: 0s - loss: 105.3361 - accuracy: 0.4978

In [ ]:
model.save("Models1/CatsAndDogs.h5")

In [ ]:
classData = ["cat" , "dog"]

saveModel = load_model("Models1/CatsAndDogs.h5")

In [ ]:
testImageData = cv2.imread("test/test/100.jpg")
imageDataFeature = imageFeatureExtrac(testImageData)/255.0
imageDataFeature = np.array([imageDataFeature])

In [ ]:
imageDataFeature.shape

In [ ]:
probability = saveModel.predict(imageDataFeature)[0]

In [ ]:
probability

In [ ]:
probaMax = probability.argmax(axis = 0)
probaMax

In [ ]:
imageDataLabels = f"{classData[probaMax]} , {probability[probaMax]*100}"

cv2.putText(testImageData , imageDataLabels , (10,35) , cv2.FONT_HERSHEY_COMPLEX , 1.0, (0,255,0) ,3)

plt.imshow(testImageData , cmap=  "gray")
plt.show